In [2]:
import sys
 
# appending a path
sys.path.append('/data/passala/Python_Utils')

import pandas as pd
import numpy as np
import CococoNet_reader
import Name_resolver
import os
import itertools
import warnings
import pickle
import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)


In [3]:
coconet_species_one = pickle.load( open( "/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/Orthogroup_based_coconets/rice_jp_merged_cococonet_pickle.p", "rb" ) )
coconet_species_two = pickle.load( open( "/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/Orthogroup_based_coconets/soybean_merged_cococonet_pickle.p", "rb" ) )

In [4]:

common_name_1 = Name_resolver.species_name_resolver('rice_jp','common')
common_name_2 = Name_resolver.species_name_resolver('soybean','common')



In [5]:
def generate_sc_table_formatted(coconet_species_one,coconet_species_two, common_name_1,common_name_2):


    
    trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(coconet_species_two.columns)]
    trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(coconet_species_two.columns)]
    double_species_1_trimmed_cococonet = trimmed_species_1_cococonet.replace(1,0)

    trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(coconet_species_one.columns)]
    trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(coconet_species_one.columns)]
    double_species_2_trimmed_cococonet = trimmed_species_2_cococonet.replace(1,0)
    cococonet_1_set_to_zero = double_species_1_trimmed_cococonet

    cococonet_2_set_to_zero = double_species_2_trimmed_cococonet

    top_10_cococonet_1genes = np.array(
        [cococonet_1_set_to_zero[c].nlargest(10).index.values for c in cococonet_1_set_to_zero]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_ccn_1_genes_dataframe = pd.DataFrame(
        data=top_10_cococonet_1genes,
        index=cococonet_1_set_to_zero.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )

    tidy_top_10 = top_10_ccn_1_genes_dataframe.melt(ignore_index= False)

    zipped_pairs = zip(tidy_top_10.index.to_list(),tidy_top_10['value'].to_list())
    binary_masked_cococonet_1 = pd.DataFrame(data = 0, columns = cococonet_1_set_to_zero.columns, index = cococonet_1_set_to_zero.index)
    #binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
    #binary_masked_cococonet.sum(axis =0)
    for row,column in zipped_pairs:
        binary_masked_cococonet_1.at[row,column] = 1

    ranked_columns_cococonet_2 = double_species_2_trimmed_cococonet.rank()

    dot_product_cococonet_2 = binary_masked_cococonet_1.dot(ranked_columns_cococonet_2)
    subtract_minimum_2 = dot_product_cococonet_2-65 # This is 11+10+9+8+7+6+5+4+3+2


    function_conservations_scores_2 = subtract_minimum_2/(subtract_minimum_2.max().max() - subtract_minimum_2.min().min())
    fc_scores_ranked_2=function_conservations_scores_2.rank()
    sc_scores_2 = fc_scores_ranked_2/fc_scores_ranked_2.shape[0]




    top_10_cococonet_2genes = np.array(
        [cococonet_2_set_to_zero[c].nlargest(10).index.values for c in cococonet_2_set_to_zero]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_ccn_2_genes_dataframe = pd.DataFrame(
        data=top_10_cococonet_2genes,
        index=cococonet_2_set_to_zero.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )

    tidy_top_10 = top_10_ccn_2_genes_dataframe.melt(ignore_index= False)

    zipped_pairs = zip(tidy_top_10.index.to_list(),tidy_top_10['value'].to_list())
    binary_masked_cococonet_2 = pd.DataFrame(data = 0, columns = cococonet_2_set_to_zero.columns, index = cococonet_2_set_to_zero.index)
    #binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
    #binary_masked_cococonet.sum(axis =0)
    for row,column in zipped_pairs:
        binary_masked_cococonet_2.at[row,column] = 1

    ranked_columns_cococonet_1 = double_species_1_trimmed_cococonet.rank()

    dot_product_cococonet_1 = binary_masked_cococonet_2.dot(ranked_columns_cococonet_1)
    subtract_minimum_1 = dot_product_cococonet_1-65 # This is 11+10+9+8+7+6+5+4+3+2


    function_conservations_scores_1 = subtract_minimum_1/(subtract_minimum_1.max().max() - subtract_minimum_1.min().min())
    fc_scores_ranked_1=function_conservations_scores_1.rank()
    sc_scores_1 = fc_scores_ranked_1/fc_scores_ranked_1.shape[0]

    transposed_1 = sc_scores_1.T

    averaged_results = (sc_scores_2+transposed_1)/2
    

    ##process average Rsults to look like table 
    output = pd.DataFrame( columns = [f'{common_name_1} and {common_name_2}'], data = averaged_results.index )
    output['Species 1 Score'] = 'compatability col'
    output['Species 2 Score'] = 'compatability col'
    output['Total Score'] = np.diag(averaged_results)

    return output


    

In [6]:
# trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(coconet_species_two.columns)]
# trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(coconet_species_two.columns)]
# double_species_1_trimmed_cococonet = trimmed_species_1_cococonet.replace(1,0)

# trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(coconet_species_one.columns)]
# trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(coconet_species_one.columns)]
# double_species_2_trimmed_cococonet = trimmed_species_2_cococonet.replace(1,0)
# cococonet_1_set_to_zero = double_species_1_trimmed_cococonet

# cococonet_2_set_to_zero = double_species_2_trimmed_cococonet

# top_10_cococonet_1genes = np.array(
#     [cococonet_1_set_to_zero[c].nlargest(10).index.values for c in cococonet_1_set_to_zero]
# )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
# top_10_ccn_1_genes_dataframe = pd.DataFrame(
#     data=top_10_cococonet_1genes,
#     index=cococonet_1_set_to_zero.index,
#     columns=[
#         "One",
#         "Two",
#         "Three",
#         "Four",
#         "Five",
#         "Six",
#         "Seven",
#         "Eight",
#         "Nine",
#         "Ten",
#     ],
# )

# tidy_top_10 = top_10_ccn_1_genes_dataframe.melt(ignore_index= False)

# zipped_pairs = zip(tidy_top_10.index.to_list(),tidy_top_10['value'].to_list())
# binary_masked_cococonet_1 = pd.DataFrame(data = 0, columns = cococonet_1_set_to_zero.columns, index = cococonet_1_set_to_zero.index)
# #binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
# #binary_masked_cococonet.sum(axis =0)
# for row,column in zipped_pairs:
#     binary_masked_cococonet_1.at[row,column] = 1

# ranked_columns_cococonet_2 = double_species_2_trimmed_cococonet.rank()

# dot_product_cococonet_2 = binary_masked_cococonet_1.dot(ranked_columns_cococonet_2)
# subtract_minimum_2 = dot_product_cococonet_2-65 # This is 11+10+9+8+7+6+5+4+3+2


# function_conservations_scores_2 = subtract_minimum_2/(subtract_minimum_2.max().max() - subtract_minimum_2.min().min())
# fc_scores_ranked_2=function_conservations_scores_2.rank()
# sc_scores_2 = fc_scores_ranked_2/fc_scores_ranked_2.shape[0]




# top_10_cococonet_2genes = np.array(
#     [cococonet_2_set_to_zero[c].nlargest(10).index.values for c in cococonet_2_set_to_zero]
# )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
# top_10_ccn_2_genes_dataframe = pd.DataFrame(
#     data=top_10_cococonet_2genes,
#     index=cococonet_2_set_to_zero.index,
#     columns=[
#         "One",
#         "Two",
#         "Three",
#         "Four",
#         "Five",
#         "Six",
#         "Seven",
#         "Eight",
#         "Nine",
#         "Ten",
#     ],
# )

# tidy_top_10 = top_10_ccn_2_genes_dataframe.melt(ignore_index= False)

# zipped_pairs = zip(tidy_top_10.index.to_list(),tidy_top_10['value'].to_list())
# binary_masked_cococonet_2 = pd.DataFrame(data = 0, columns = cococonet_2_set_to_zero.columns, index = cococonet_2_set_to_zero.index)
# #binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
# #binary_masked_cococonet.sum(axis =0)
# for row,column in zipped_pairs:
#     binary_masked_cococonet_2.at[row,column] = 1

# ranked_columns_cococonet_1 = double_species_1_trimmed_cococonet.rank()

# dot_product_cococonet_1 = binary_masked_cococonet_2.dot(ranked_columns_cococonet_1)
# subtract_minimum_1 = dot_product_cococonet_1-65 # This is 11+10+9+8+7+6+5+4+3+2


# function_conservations_scores_1 = subtract_minimum_1/(subtract_minimum_1.max().max() - subtract_minimum_1.min().min())
# fc_scores_ranked_1=function_conservations_scores_1.rank()
# sc_scores_1 = fc_scores_ranked_1/fc_scores_ranked_1.shape[0]

# transposed_1 = sc_scores_1.T

# averaged_results = (sc_scores_2+transposed_1)/2

In [8]:

# common_name_1 = Name_resolver.species_name_resolver('rice_jp','common')
# common_name_2 = Name_resolver.species_name_resolver('apple','common')





# ## Read In Cococonets 
# coconet_species_one = coconet_1
# coconet_species_two = coconet_2







# #Identify Pairs for evaluation

# pairs_to_keep = list(set(coconet_species_one.index.to_list()) & set(coconet_species_two.index.to_list()))

# all_pairs_to_evaluate_for_functional_conservation = pd.DataFrame(columns = [f'{common_name_1} and {common_name_2}'],data= pairs_to_keep)


# all_pairs_to_evaluate_for_functional_conservation['Species 1 Score'] = np.nan
# all_pairs_to_evaluate_for_functional_conservation['Species 2 Score'] = np.nan


# ## Trim cococonets to match


# trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(pairs_to_keep)]
# trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(pairs_to_keep)]
# double_species_1_trimmed_cococonet = trimmed_species_1_cococonet.replace(1,0)

# trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(pairs_to_keep)]
# trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(pairs_to_keep)]
# double_species_2_trimmed_cococonet = trimmed_species_2_cococonet.replace(1,0)


In [9]:
# ## Rank
# species_1_cococonet_ranked = trimmed_species_1_cococonet.rank()
# species_2_cococonet_ranked = trimmed_species_2_cococonet.rank()

# #Do top 10 Genes
# top_10_species_1_genes = np.array(
#     [double_species_1_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_1_trimmed_cococonet.T]
# )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
# top_10_species_1_genes_dataframe = pd.DataFrame(
#     data=top_10_species_1_genes,
#     index=double_species_1_trimmed_cococonet.index,
#     columns=[
#         "One",
#         "Two",
#         "Three",
#         "Four",
#         "Five",
#         "Six",
#         "Seven",
#         "Eight",
#         "Nine",
#         "Ten",
#     ],
# )

# #Convert 
# top_10_species_1_genes_as_species_2 = top_10_species_1_genes_dataframe

# # Get genes for checking 
# have_species_1_pairs = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[f'{common_name_1} and {common_name_2}'].isin(top_10_species_1_genes_as_species_2.index)]
# trimmed_all_gene_pairs_for_fc = have_species_1_pairs.loc[have_species_1_pairs[f'{common_name_1} and {common_name_2}'].isin(trimmed_species_2_cococonet.index)]
# trimmed_all_gene_pairs_for_fc = trimmed_all_gene_pairs_for_fc.reset_index(drop = True)

# # Get values in species 2 
# for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
#     current_species_1_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
#     current_species_2_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
#     finger_print_genes = top_10_species_1_genes_as_species_2.loc[current_species_1_gene].to_list()
#     gene_ranks_in_species_2 = species_2_cococonet_ranked.loc[species_2_cococonet_ranked.index.isin(finger_print_genes), current_species_2_gene]
#     avg_rank_in_species_2 = gene_ranks_in_species_2.mean()
#     index_from_pairs = two_genes[0]
#     trimmed_all_gene_pairs_for_fc.at[index_from_pairs, 'Species 1 Score'] = avg_rank_in_species_2

# #Repeat for Species 2 

# top_10_species_2_genes = np.array(
#     [double_species_2_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_2_trimmed_cococonet.T]
# )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
# top_10_species_2_genes_dataframe = pd.DataFrame(
#     data=top_10_species_2_genes,
#     index=double_species_2_trimmed_cococonet.index,
#     columns=[
#         "One",
#         "Two",
#         "Three",
#         "Four",
#         "Five",
#         "Six",
#         "Seven",
#         "Eight",
#         "Nine",
#         "Ten",
#     ],
# )


# #convert 
# top_10_species_2_genes_as_species_1 = top_10_species_2_genes_dataframe


# # Get values in species 1 
# for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
#     current_species_1_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
#     current_species_2_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
#     finger_print_genes = top_10_species_2_genes_as_species_1.loc[current_species_2_gene].to_list()
#     gene_ranks_in_species_1 = species_1_cococonet_ranked.loc[species_1_cococonet_ranked.index.isin(finger_print_genes), current_species_1_gene]
#     avg_rank_in_species_1 = gene_ranks_in_species_1.mean()
#     index_from_pairs = two_genes[0]
#     trimmed_all_gene_pairs_for_fc.loc[index_from_pairs, 'Species 2 Score'] = avg_rank_in_species_1

# #Caluclate Divisors 
# Number_of_species_1_genes = len(top_10_species_1_genes_as_species_2)
# Number_of_species_2_genes = len(top_10_species_2_genes_as_species_1)


# species_1_score_divisor = Number_of_species_2_genes - 4.5
# species_2_score_divisor = Number_of_species_1_genes-4.5

# #Divide and Average 
# trimmed_all_gene_pairs_for_fc['Species 1 Score'] = trimmed_all_gene_pairs_for_fc['Species 1 Score']/species_1_score_divisor
# trimmed_all_gene_pairs_for_fc['Species 2 Score'] = trimmed_all_gene_pairs_for_fc['Species 2 Score']/species_2_score_divisor
# trimmed_all_gene_pairs_for_fc['Total Score'] = trimmed_all_gene_pairs_for_fc[['Species 1 Score','Species 2 Score']].mean(axis = 1)

In [10]:
def Calculate_Score_list_for_thresholding(coconet_1, coconet_2, Species_1 = 'rice',Species_2 = 'maize', ):

    import pandas as pd
    import itertools

    common_name_1 = Name_resolver.species_name_resolver(Species_1,'common')
    common_name_2 = Name_resolver.species_name_resolver(Species_2,'common')





    ## Read In Cococonets 
    coconet_species_one = coconet_1
    coconet_species_two = coconet_2







    #Identify Pairs for evaluation

    pairs_to_keep = list(set(coconet_species_one.index.to_list()) & set(coconet_species_two.index.to_list()))

    all_pairs_to_evaluate_for_functional_conservation = pd.DataFrame(columns = [f'{common_name_1} and {common_name_2}'],data= pairs_to_keep)


    all_pairs_to_evaluate_for_functional_conservation['Species 1 Score'] = np.nan
    all_pairs_to_evaluate_for_functional_conservation['Species 2 Score'] = np.nan


    ## Trim cococonets to match


    trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(pairs_to_keep)]
    trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(pairs_to_keep)]
    double_species_1_trimmed_cococonet = trimmed_species_1_cococonet.replace(1,0)

    trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(pairs_to_keep)]
    trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(pairs_to_keep)]
    double_species_2_trimmed_cococonet = trimmed_species_2_cococonet.replace(1,0)



    ## Rank
    species_1_cococonet_ranked = trimmed_species_1_cococonet.rank()
    species_2_cococonet_ranked = trimmed_species_2_cococonet.rank()

    #Do top 10 Genes
    top_10_species_1_genes = np.array(
        [double_species_1_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_1_trimmed_cococonet.T]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_species_1_genes_dataframe = pd.DataFrame(
        data=top_10_species_1_genes,
        index=double_species_1_trimmed_cococonet.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )

    #Convert 
    top_10_species_1_genes_as_species_2 = top_10_species_1_genes_dataframe

    # Get genes for checking 
    have_species_1_pairs = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[f'{common_name_1} and {common_name_2}'].isin(top_10_species_1_genes_as_species_2.index)]
    trimmed_all_gene_pairs_for_fc = have_species_1_pairs.loc[have_species_1_pairs[f'{common_name_1} and {common_name_2}'].isin(trimmed_species_2_cococonet.index)]
    trimmed_all_gene_pairs_for_fc = trimmed_all_gene_pairs_for_fc.reset_index(drop = True)

    # Get values in species 2 
    for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
        current_species_1_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
        current_species_2_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
        finger_print_genes = top_10_species_1_genes_as_species_2.loc[current_species_1_gene].to_list()
        gene_ranks_in_species_2 = species_2_cococonet_ranked.loc[species_2_cococonet_ranked.index.isin(finger_print_genes), current_species_2_gene]
        avg_rank_in_species_2 = gene_ranks_in_species_2.mean()
        index_from_pairs = two_genes[0]
        trimmed_all_gene_pairs_for_fc.at[index_from_pairs, 'Species 1 Score'] = avg_rank_in_species_2

    #Repeat for Species 2 

    top_10_species_2_genes = np.array(
        [double_species_2_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_2_trimmed_cococonet.T]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_species_2_genes_dataframe = pd.DataFrame(
        data=top_10_species_2_genes,
        index=double_species_2_trimmed_cococonet.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )


    #convert 
    top_10_species_2_genes_as_species_1 = top_10_species_2_genes_dataframe


    # Get values in species 1 
    for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
        current_species_1_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
        current_species_2_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
        finger_print_genes = top_10_species_2_genes_as_species_1.loc[current_species_2_gene].to_list()
        gene_ranks_in_species_1 = species_1_cococonet_ranked.loc[species_1_cococonet_ranked.index.isin(finger_print_genes), current_species_1_gene]
        avg_rank_in_species_1 = gene_ranks_in_species_1.mean()
        index_from_pairs = two_genes[0]
        trimmed_all_gene_pairs_for_fc.loc[index_from_pairs, 'Species 2 Score'] = avg_rank_in_species_1

    #Caluclate Divisors 
    Number_of_species_1_genes = len(top_10_species_1_genes_as_species_2)
    Number_of_species_2_genes = len(top_10_species_2_genes_as_species_1)


    species_1_score_divisor = Number_of_species_2_genes - 4.5
    species_2_score_divisor = Number_of_species_1_genes-4.5

    #Divide and Average 
    trimmed_all_gene_pairs_for_fc['Species 1 Score'] = trimmed_all_gene_pairs_for_fc['Species 1 Score']/species_1_score_divisor
    trimmed_all_gene_pairs_for_fc['Species 2 Score'] = trimmed_all_gene_pairs_for_fc['Species 2 Score']/species_2_score_divisor
    trimmed_all_gene_pairs_for_fc['Total Score'] = trimmed_all_gene_pairs_for_fc[['Species 1 Score','Species 2 Score']].mean(axis = 1)
        
    return trimmed_all_gene_pairs_for_fc

In [11]:
Calculate_Score_list_for_thresholding(coconet_species_one,coconet_species_two,'rice_jp','soybean')

,rice_jp and soybean,Species 1 Score,Species 2 Score,Total Score
0,110263at3193,0.711713,0.833536,0.772624
1,128468at3193,0.896613,0.785427,0.841020
2,882547at3193,0.704806,0.792114,0.748460
3,125951at3193,0.876904,0.829088,0.852996
4,678282at3193,0.755169,0.683986,0.719577
...,...,...,...,...
6834,683427at3193,0.381125,0.665930,0.523528
6835,878363at3193,0.800907,0.818363,0.809635
6836,123940at3193,0.976619,0.931304,0.953962
6837,875771at3193,0.562719,0.772258,0.667488


In [12]:
generate_sc_table_formatted(coconet_species_one,coconet_species_two,'rice_jp','soybean')

,rice_jp and soybean,Species 1 Score,Species 2 Score,Total Score
0,10092at3193,compatability col,compatability col,0.889823
1,10147at3193,compatability col,compatability col,0.873666
2,10228at3193,compatability col,compatability col,0.871034
3,10238at3193,compatability col,compatability col,0.662195
4,10525at3193,compatability col,compatability col,0.656346
...,...,...,...,...
6834,9789at3193,compatability col,compatability col,0.713189
6835,9828at3193,compatability col,compatability col,0.934566
6836,9877at3193,compatability col,compatability col,0.955768
6837,9919at3193,compatability col,compatability col,0.969550


In [13]:
species_to_run_on = pd.read_csv('/data/passala/Generated_Tables/Reference_tables/species_for_running_cross_ortho_analysis.csv')


In [14]:
list_of_species_to_run_on = species_to_run_on['Common Name'].to_list()


In [15]:
all_coconets = {}
for species in tqdm.tqdm(list_of_species_to_run_on):
    current_net = pickle.load( open( f"/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/Orthogroups_at_eukaryote_level/Orthogroup_cococonet_eukaryote/{species}_merged_cococonet_pickle.p", "rb" ) )
    all_coconets[species] = current_net

100%|██████████| 15/15 [00:04<00:00,  3.13it/s]


In [16]:
all_combos_to_run_on = list(itertools.combinations(list_of_species_to_run_on,2))


In [17]:
len(all_combos_to_run_on)


105

In [18]:
generated_files = os.listdir('/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/1_1_ortho_FC_tables')

In [19]:
len(generated_files)

105

In [20]:
for combo in tqdm.tqdm(all_combos_to_run_on):
    species_1 = combo[0]
    species_2 = combo[1]
    coconet_spec_1 = all_coconets[species_1]
    coconet_spec_2 = all_coconets[species_2]
    
    score_list = generate_sc_table_formatted(coconet_spec_1, coconet_spec_2, species_1,species_2)
    score_list.to_csv(f'/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/Orthogroups_at_eukaryote_level/1_1_ortho_SC_tables_eukaryote/{species_1}_{species_2}_NM.csv')


100%|██████████| 105/105 [1:19:47<00:00, 45.60s/it]
